In [22]:
# Importing packages
import os
import json
import pandas as pd
import traceback

In [23]:
from langchain.chat_models import ChatOpenAI

In [24]:
#To utilise the environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
KEY = os.getenv("OPEN_API_KEY")

In [26]:
#Creating llm model of gpt 3.5 in to the system
llm= ChatOpenAI(openai_api_key= KEY, model_name= 'gpt-3.5-turbo', temperature= 0.5)

In [27]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [28]:
# Response json is the format needed by the model
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [29]:
#Template given to the model
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [30]:
#Creating the prompt using prompt template by giving input variables and template

quiz_generation_prompt= PromptTemplate(
    input_variables= ["text", "number", "subject", "tone", "response_json"],
    template= TEMPLATE
)

In [31]:
#Creating first component of the chain for generating the quiz
quiz_chain= LLMChain(llm=llm, prompt= quiz_generation_prompt, output_key='quiz', verbose= True)

In [32]:
#Template to get complete analysis of the quiz
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [33]:
#Creating prompt template using input variables and prompt
quiz_evaluation_prompt= PromptTemplate(input_variables=['subject', 'quiz'], template= TEMPLATE2)

In [34]:
#Creating LLM chain component for quiz evaluation
review_chain= LLMChain(llm=llm, prompt= quiz_evaluation_prompt, output_key= "review", verbose= True)

In [35]:
#Combining both quiz chain and review chain using sequential chain function

generate_evaluate_chain= SequentialChain(chains= [quiz_chain, review_chain], input_variables= ['text', 'number', "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True,)

In [36]:
# Opening the text file
with open(r"D:\ai-pro\project\data.txt", 'r') as file:
    TEXT= file.read()

In [37]:
#serialize the python dictionary into a JSON-formatted string

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
#Intializing the variables
NUMBER=3 
SUBJECT="Machine Learning"
TONE="simple"

In [39]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:What is ML?
Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.

How does machine learning work?
UC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.
An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.
A Model Optimization Process: If the model can fit better to the data points i

In [40]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1427
Prompt Tokens:946
Completion Tokens:481
Total Cost:0.002381


In [43]:
#Sample response
response

{'text': 'What is ML?\nMachine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.\n\nHow does machine learning work?\nUC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.\n\nA Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.\nAn Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.\nA Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known 

In [41]:
quiz=response.get("quiz")
quiz=json.loads(quiz)

In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz_table_data

[{'MCQ': 'What is the main focus of machine learning?',
  'Choices': 'a: Using data and algorithms to enable AI to imitate human learning | b: Creating robots with human-like intelligence | c: Developing software applications for businesses | d: Studying the history of artificial intelligence',
  'Correct': 'a'},
 {'MCQ': 'What does the error function in a machine learning algorithm do?',
  'Choices': 'a: Makes a prediction based on input data | b: Evaluates the prediction of the model | c: Optimizes the model weights | d: Adjusts the threshold of accuracy',
  'Correct': 'b'},
 {'MCQ': 'What is the final goal of the model optimization process in machine learning?',
  'Choices': 'a: To increase the number of data points in the training set | b: To reduce the discrepancy between known examples and model estimates | c: To stop the algorithm from updating weights | d: To make the algorithm repeat the process indefinitely',
  'Correct': 'b'}]